In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

import transformers
import tokenizers

from new_semantic_parsing import EncoderDecoderWPointerModel

In [13]:
ENCODER_NAME = 'distilbert-base-uncased'
OUT_VOCAB_SIZE = 100
HIDDEN = 768

BS = 3
SRC_LEN = 5
TGT_LEN = 7

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(ENCODER_NAME, use_fast=True)
encoder = transformers.AutoModel.from_pretrained(ENCODER_NAME)

# BERTConfig is a generic transformer and is only decoder Transformers support by now
decoder_config = transformers.BertConfig(
    vocab_size=OUT_VOCAB_SIZE,
    hidden_size=HIDDEN,
    is_decoder=True,  # adds cross-attention modules and enables causal masking
)

decoder = transformers.BertModel(decoder_config)

In [7]:
model = EncoderDecoderWPointerModel(encoder, decoder)

In [21]:
x_enc = torch.randint(0, tokenizer.vocab_size, size=(BS, SRC_LEN))
x_dec = torch.randint(0, OUT_VOCAB_SIZE, size=(BS, TGT_LEN))

out = model(input_ids=x_enc, decoder_input_ids=x_dec)
combined_logits, decoder_hidden, decoder_pooled, encoder_hidden = out

assert combined_logits.shape == (BS, TGT_LEN, OUT_VOCAB_SIZE + SRC_LEN)

In [23]:
decoder_pooled.shape

torch.Size([3, 768])